In [13]:

from dataclasses import dataclass
from typing import List

# clases abstractas

@dataclass
class Node:
    pass

@dataclass
class Stmt(Node):
    pass

@dataclass
class Expr(Node):
    pass

# clases concretas del AST
@dataclass
class Programan(Stmt):
  stmt : List[Stmt]

    

@dataclass
class Assignment(Stmt):
    var : Expr
    exp : Expr


@dataclass
class Binop(Expr):
    op    : str
    left  : Expr
    right : Expr

@dataclass
class Unaryop(Expr):
    op  : str
    exp : Expr

@dataclass
class Variable(Expr):
    name : str

@dataclass
class Number(Expr):
    value : int

@dataclass
class Let(Stmt):
    var : Expr
    expr : Expr

@dataclass
class Read(Stmt):
    var : List[Expr]


@dataclass
class Data(Stmt):
    expr : List
  
@dataclass
class Print(Stmt):
    expr : List[Expr]

@dataclass
class If(Stmt):
    expr : Expr

@dataclass
class Goto(Stmt):
    line : int


@dataclass
class For(Stmt):
  vars : Expr
  init : Expr
  to : Expr
  step :Expr = None

@dataclass
class Next(Stmt):
  vars : Expr

@dataclass
class Gosub(Stmt):
    line : int

 

    






In [10]:
!pip install multimethod
from multimethod import multimeta
import graphviz as gv

# Patron Visitor

class Visitor(metaclass=multimeta):
    pass


class Dot(Visitor):
    
    _node_defaults = {
        'color' : 'lightblue2',
        'style' : 'filled'
    }
    
    def __init__(self):
        self.id = 0
        self.dot = gv.Graph(name='AST',node_attr=self._node_defaults)

    def name(self):
        self.id += 1
        return 'n%02d' % self.id

    def visit(self, n: Assignment):
        name = self.name()
        self.dot.node(name, label='=')
        var = self.visit(n.var)
        exp = self.visit(n.exp)
        self.dot.edge(name, var, label='var')
        self.dot.edge(name, exp, label='exp')
        return name

    def visit(self, n: Binop):
        name = self.name()
        self.dot.node(name, label=n.op)
        left  = self.visit(n.left)
        right = self.visit(n.right)
        self.dot.edge(name, left)
        self.dot.edge(name, right)
        return name

    def visit(self, n: Unaryop):
        name = self.name()
        self.dot.node(name, label=n.op)
        exp = self.visit(n.exp)
        self.dot.edge(name, exp)
        return name

    def visit(self, n: Variable):
        name = self.name()
        self.dot.node(name, label=n.name, shape='box')
        return name


    def visit(self, n: Number):
        name = self.name()
        self.dot.node(name, label=str(n.value), shape='box')
        return name

NameError: name 'Assignment' is not defined

In [6]:
# Analizador lexico
!pip install sly
import sly

class Lexer(sly.Lexer):
    
    tokens = {
        NAME, NUMBER,LET, READ, DATA, PRINT, GOTO, IF, THEN, FOR, NEXT, TO, STEP, END,
        STOP, DEF, GOSUB, DIM, REM, RETURN, RUN, LIST, NEW,
    }
    literals = '+-*/^=()'
    
    ignore = ' \t\r'
    
    NAME = r'[A-Z][0-9]?'
   
    @_("\d+")
    def NUMBER(self, t):
        t.value = int(t.value)
        return t


    
    @_("\n+")
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')
    
    def error(self, t):
        print(f"{t.lineno} - Caracter ilegal '{t.value[0]}'")
        self.index += 1

In [7]:
class Parser(sly.Parser):
    
    #debugfile = "calc.txt"
    
    tokens = Lexer.tokens
    
    precedence = (
        ('left', '+', '-'),
        ('left', '*', '/'),
        ('right', '^'),
    )
    
    @_("variable '=' expr")
    def stmt(self, p):
        return Assignment(p.variable, p.expr)

    @_("expr")
    def stmt(self, p):
        return p.expr

    @_("expr '+' expr",
       "expr '-' expr",
       "expr '*' expr",
       "expr '/' expr",
       "expr '^' expr")
    def expr(self, p):
        return Binop(p[1], p.expr0, p.expr1)

    @_("'-' expr")
    def expr(self, p):
        return Unaryop(p[0], p.expr)
    
    @_("'(' expr ')'")
    def expr(self, p):
        return p.expr
    
    @_("variable", "number")
    def expr(self, p):
        return p[0]

    @_("NAME")
    def variable(self, p):
        return Variable(p.NAME)
    
    @_("NUMBER")
    def number(self, p):
        return Number(p.NUMBER)
  
    def error(self, p):
        if p:
            print(f"Error de sintaxis en token: '{p.type}'")
            self.errok()
        else:
            print("Error de sintaxis en EOF")

    
    

In [5]:
from graphviz import Source

Source.from_file('test/round-table.gv')


Error: <stdin>: syntax error in line 3 near '''


CalledProcessError: Command '['dot', '-Kdot', '-Tsvg']' returned non-zero exit status 1. [stderr: b"Error: <stdin>: syntax error in line 3 near '''\r\n"]